1st 사용 방법

In [52]:
import joblib
import pandas as pd
import time
from sklearn.metrics.pairwise import cosine_similarity
from simple_preprocessor import SimplePreprocessor

# 📁 모델 경로
load_dir = "AI_Model_2"

# ✅ 모델 관련 데이터 로딩
print("📦 모델 및 데이터 로딩 중...")
start = time.time()
df_model = pd.read_pickle(f"{load_dir}/df_model.pkl")
vectorizer = joblib.load(f"{load_dir}/searchname_vectorizer.pkl")
preprocessor = joblib.load(f"{load_dir}/SimplePreprocessor.pkl")
valid_combinations = joblib.load(f"{load_dir}/valid_combinations.pkl")

# 계층별 모델, 벡터, 인코더 로드
required_cols = ['L1 NAME', 'L2 NAME', 'L3 NAME', 'L4 NAME']
hierarchical_models = {}
hierarchical_label_encoders = {}

for col in required_cols:
    print(f"📂 {col} 로딩 중...")
    model_path = f"{load_dir}/{col}_model.pkl"
    vec_path = f"{load_dir}/{col}_vectorizer.pkl"
    enc_path = f"{load_dir}/{col}_label_encoder.pkl"

    model = joblib.load(model_path)
    vec = joblib.load(vec_path)
    enc = joblib.load(enc_path)

    hierarchical_models[col] = (model, vec)
    hierarchical_label_encoders[col] = enc

print(f"✅ 모든 로딩 완료! (소요 시간: {round(time.time() - start, 2)}초)")

# ✅ 예측 + 보정 함수 정의
def correct_prediction(l1, l2, l3, l4, search_input):
    combo_key = f"{l1} | {l2} | {l3} | {l4}"
    if combo_key in valid_combinations:
        return {
            "L1 NAME": l1, "L2 NAME": l2, "L3 NAME": l3, "L4 NAME": l4,
            "CORRECTED": False, "MATCHED_KEY": combo_key
        }

    input_vec = vectorizer.transform([search_input.upper()])
    candidate_vecs = vectorizer.transform(df_model["SEARCH_NAME"])
    sims = cosine_similarity(input_vec, candidate_vecs).flatten()
    best_idx = sims.argmax()
    best_row = df_model.iloc[best_idx]

    return {
        "L1 NAME": best_row["L1 NAME"],
        "L2 NAME": best_row["L2 NAME"],
        "L3 NAME": best_row["L3 NAME"],
        "L4 NAME": best_row["L4 NAME"],
        "CORRECTED": True,
        "MATCHED_KEY": best_row["COMBO_KEY"],
        "SIMILARITY": round(sims[best_idx], 3)
    }

def predict_item_name(raw_name):
    normalized = preprocessor.preprocess_normalized(raw_name)
    context = normalized
    prediction = {}

    for col in required_cols:
        model, vec = hierarchical_models[col]
        input_vec = vec.transform([context])
        label_id = model.predict(input_vec)[0]
        label = hierarchical_label_encoders[col].inverse_transform([label_id])[0]
        prediction[col] = label
        context += " " + label

    input_vec = vectorizer.transform([normalized])
    candidate_vecs = vectorizer.transform(df_model["ITEM_NAME"])
    sims = cosine_similarity(input_vec, candidate_vecs).flatten()
    best_idx = sims.argmax()
    l5 = df_model.iloc[best_idx]["L5 NAME (SPEC)"]
    prediction["L5 NAME (SPEC)"] = l5

    correction = correct_prediction(
        prediction["L1 NAME"],
        prediction["L2 NAME"],
        prediction["L3 NAME"],
        prediction["L4 NAME"],
        search_input=raw_name
    )
    if correction["CORRECTED"]:
        print(f"⚠️ 조합 보정됨: 유사도 {correction['SIMILARITY']}")
        for level in required_cols:
            prediction[level] = correction[level]

    print(f"🔍 L5는 분류 대신 유사도 기반 추천으로 예측되었습니다 (유사도: {round(sims[best_idx], 3)})")
    return prediction

def recommend_similar_items(input_name, top_n=5):
    query = preprocessor.preprocess_normalized(input_name)
    input_vec = vectorizer.transform([query])

    predicted = predict_item_name(input_name)
    filtered_df = df_model[
        (df_model["L1 NAME"] == predicted["L1 NAME"]) &
        (df_model["L2 NAME"] == predicted["L2 NAME"]) &
        (df_model["L3 NAME"] == predicted["L3 NAME"]) &
        (df_model["L4 NAME"] == predicted["L4 NAME"])
    ]

    if filtered_df.empty:
        return []

    candidate_vecs = vectorizer.transform(filtered_df["SEARCH_NAME"])
    similarities = cosine_similarity(input_vec, candidate_vecs).flatten()
    top_indices = similarities.argsort()[::-1]

    results = []
    for idx in top_indices:
        row = filtered_df.iloc[idx]
        sim = similarities[idx]
        if sim <= 0:
            continue
        results.append({
            "SIMILAR_ITEM_NAME": row["ITEM_NAME"],
            "SIMILARITY_SCORE": round(sim, 3),
            "P CODE": row["P CODE"]
        })
        if len(results) >= top_n:
            break
    return results

def recommend_global_similar_items(input_name, top_n=5):
    query = preprocessor.preprocess_normalized(input_name)
    input_vec = vectorizer.transform([query])
    candidate_vecs = vectorizer.transform(df_model["SEARCH_NAME"])

    similarities = cosine_similarity(input_vec, candidate_vecs).flatten()
    top_indices = similarities.argsort()[::-1]

    results = []
    for idx in top_indices:
        row = df_model.iloc[idx]
        results.append({
            "SIMILAR_ITEM_NAME": row["SEARCH_NAME"],
            "SIMILARITY_SCORE": round(similarities[idx], 3),
            "P CODE": row["P CODE"],
            "L1 NAME": row["L1 NAME"],
            "L2 NAME": row["L2 NAME"],
            "L3 NAME": row["L3 NAME"],
            "L4 NAME": row["L4 NAME"],
            "L5 NAME (SPEC)": row["L5 NAME (SPEC)"]
        })
        if len(results) >= top_n:
            break

    return results

# ✅ 실행 함수 with 루프 지원
def run_model():
    while True:
        user_input = input("🔍 품명을 입력하세요 (종료하려면 'exit'): ").strip()
        if not user_input:
            print("❗ 품명이 비어 있습니다.")
            continue
        if user_input.lower() in ["exit", "quit"]:
            print("👋 종료합니다.")
            break

        print("\n🕒 예측 시작...")
        t_start = time.time()

        prediction = predict_item_name(user_input)

        combo_key = f"{prediction['L1 NAME']} | {prediction['L2 NAME']} | {prediction['L3 NAME']} | {prediction['L4 NAME']}"
        if combo_key not in valid_combinations:
            input_vec = vectorizer.transform([user_input.upper()])
            sims = cosine_similarity(input_vec, vectorizer.transform(df_model["SEARCH_NAME"])).flatten()
            best_sim = round(sims.max(), 3)
            print(f"\n⚠️ 예측된 L1~L4 조합이 존재하지 않아 보정되었습니다 (유사도: {best_sim})")

        input_vec = vectorizer.transform([preprocessor.preprocess_normalized(user_input)])
        candidate_vecs = vectorizer.transform(df_model["ITEM_NAME"])
        l5_sim = cosine_similarity(input_vec, candidate_vecs).flatten().max()
        print(f"🔍 L5는 분류 대신 유사도 기반 추천으로 예측되었습니다 (유사도: {round(l5_sim, 3)})")

        global_similar = recommend_global_similar_items(user_input)
        if global_similar:
            print("\n🔎 전역 유사 품목 Top 5:")
            for rec in global_similar:
                print(f"\n▶ 품명: {rec['SIMILAR_ITEM_NAME']} (유사도: {round(rec['SIMILARITY_SCORE'] * 100, 1)}%)")
                print(f"   🔹 P CODE: {rec['P CODE']}")
                print(f"   - L1: {rec['L1 NAME']}")
                print(f"   - L2: {rec['L2 NAME']}")
                print(f"   - L3: {rec['L3 NAME']}")
                print(f"   - L4: {rec['L4 NAME']}")
                print(f"   - L5: {rec['L5 NAME (SPEC)']}")

        pred_string = " ".join([prediction.get(k, '') for k in required_cols + ['L5 NAME (SPEC)']])
        sim_score = cosine_similarity(vectorizer.transform([user_input.upper()]), vectorizer.transform([pred_string.upper()]))[0][0]
        print(f"\n📊 예측 신뢰도 (입력 vs 예측): {round(sim_score * 100, 1)}%")

        print("\n🎯 예측된 분류:")
        print(f"📁 L1 (대분류): {prediction['L1 NAME']}")
        print(f"📂 L2 (중분류): {prediction['L2 NAME']}")
        print(f"📄 L3 (소분류): {prediction['L3 NAME']}")
        print(f"🔹 L4 (세분류): {prediction['L4 NAME']}")
        print(f"🧷 L5 (상세/스펙): {prediction['L5 NAME (SPEC)']}")

        matched = df_model[
            (df_model["L1 NAME"] == prediction["L1 NAME"]) &
            (df_model["L2 NAME"] == prediction["L2 NAME"]) &
            (df_model["L3 NAME"] == prediction["L3 NAME"]) &
            (df_model["L4 NAME"] == prediction["L4 NAME"]) &
            (df_model["L5 NAME (SPEC)"] == prediction["L5 NAME (SPEC)"])
        ]
        if not matched.empty:
            print(f"- 📦 추천 P CODE (기존): {matched.iloc[0]['P CODE']}")
        else:
            print("- 📭 해당 조합에 대한 기존 P CODE 없음")

        similar_items = recommend_similar_items(user_input)
        if similar_items:
            print("\n🔎 유사한 품목 Top 5:")
            for rec in similar_items:
                print(f"\n▶ 품명: {rec['SIMILAR_ITEM_NAME']} (유사도: {round(rec['SIMILARITY_SCORE'] * 100, 1)}%)")
                print(f"   🔹 P CODE: {rec['P CODE']}")
        else:
            print("📭 유사한 품명이 존재하지 않습니다.")

        print(f"\n✅ 총 소요 시간: {round(time.time() - t_start, 2)}초")

# ▶️ 실행
if __name__ == "__main__":
    run_model()


📦 모델 및 데이터 로딩 중...
📂 L1 NAME 로딩 중...
📂 L2 NAME 로딩 중...
📂 L3 NAME 로딩 중...
📂 L4 NAME 로딩 중...
✅ 모든 로딩 완료! (소요 시간: 1.75초)

🕒 예측 시작...


c:\Users\정하민\Desktop\엠투코리아 모델\bert_cuda_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


⚠️ 조합 보정됨: 유사도 0.732
🔍 L5는 분류 대신 유사도 기반 추천으로 예측되었습니다 (유사도: 0.894)
🔍 L5는 분류 대신 유사도 기반 추천으로 예측되었습니다 (유사도: 0.894)

🔎 전역 유사 품목 Top 5:

▶ 품명: LIQUOR WHISKY BALLANTINES BALLANTINES FINEST 1LX12BTL (유사도: 73.2%)
   🔹 P CODE: PAWHB1BG02
   - L1: LIQUOR
   - L2: WHISKY
   - L3: BALLANTINES
   - L4: BALLANTINES FINEST
   - L5: 1LX12BTL

▶ 품명: LIQUOR WHISKY BALLANTINES BALLANTINES 30Y 700MLX6BTL (유사도: 73.2%)
   🔹 P CODE: PAWHB1BC03
   - L1: LIQUOR
   - L2: WHISKY
   - L3: BALLANTINES
   - L4: BALLANTINES 30Y
   - L5: 700MLX6BTL

▶ 품명: LIQUOR WHISKY BALLANTINES BALLANTINES 21Y 700MLX12BTL (유사도: 73.2%)
   🔹 P CODE: PAWHB1B602
   - L1: LIQUOR
   - L2: WHISKY
   - L3: BALLANTINES
   - L4: BALLANTINES 21Y
   - L5: 700MLX12BTL

▶ 품명: LIQUOR WHISKY BALLANTINES BALLANTINES 17Y 700MLX12BTL (유사도: 73.2%)
   🔹 P CODE: PAWHB1B402
   - L1: LIQUOR
   - L2: WHISKY
   - L3: BALLANTINES
   - L4: BALLANTINES 17Y
   - L5: 700MLX12BTL

▶ 품명: LIQUOR WHISKY BALLANTINES BALLANTINES 12Y 1LX12BTL (유사도: 73.2%)
   🔹 P CODE: 

c:\Users\정하민\Desktop\엠투코리아 모델\bert_cuda_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


⚠️ 조합 보정됨: 유사도 0.732
🔍 L5는 분류 대신 유사도 기반 추천으로 예측되었습니다 (유사도: 0.894)

🔎 유사한 품목 Top 5:

▶ 품명: BALLANTINES BALLANTINES 12Y 1LX12BTL (유사도: 73.2%)
   🔹 P CODE: PAWHB1B202

▶ 품명: BALLANTINES BALLANTINES 12Y 1L (유사도: 70.8%)
   🔹 P CODE: PAWHB1B201

▶ 품명: BALLANTINES BALLANTINES 12Y 750MLX12BTL (유사도: 68.8%)
   🔹 P CODE: PAWHB1B203

✅ 총 소요 시간: 4.96초

🕒 예측 시작...


...

⚠️ 조합 보정됨: 유사도 0.776
🔍 L5는 분류 대신 유사도 기반 추천으로 예측되었습니다 (유사도: 0.877)
🔍 L5는 분류 대신 유사도 기반 추천으로 예측되었습니다 (유사도: 0.877)

🔎 전역 유사 품목 Top 5:

▶ 품명: HAND TOOL HOSE BAND / PIPE CLAMP STRAUB COUPLINGS PIPE COUPLING STRAUB GRIP TYPE   RUBBER EPDM SLEEVE 50A (유사도: 77.6%)
   🔹 P CODE: SCASS2P205
   - L1: HAND TOOL
   - L2: HOSE BAND / PIPE CLAMP
   - L3: STRAUB COUPLINGS
   - L4: PIPE COUPLING STRAUB GRIP-TYPE,  RUBBER EPDM SLEEVE
   - L5: 50A

▶ 품명: HAND TOOL HOSE BAND / PIPE CLAMP STRAUB COUPLINGS PIPE COUPLING STRAUB GRIP TYPE   RUBBER EPDM SLEEVE 40A (유사도: 77.5%)
   🔹 P CODE: SCASS2P204
   - L1: HAND TOOL
   - L2: HOSE BAND / PIPE CLAMP
   - L3: STRAUB COUPLINGS
   - L4: PIPE COUPLING STRAUB GRIP-TYPE,  RUBBER EPDM SLEEVE
   - L5: 40A

▶ 품명: HAND TOOL HOSE BAND / PIPE CLAMP STRAUB COUPLINGS PIPE COUPLING STRAUB GRIP TYPE   RUBBER EPDM SLEEVE 25A (유사도: 77.5%)
   🔹 P CODE: SCASS2P202
   - L1: HAND TOOL
   - L2: HOSE BAND / PIPE CLAMP
   - L3: STRAUB COUPLINGS
   - L4: PIPE COUPLING STRAUB GRIP-TYPE, 

c:\Users\정하민\Desktop\엠투코리아 모델\bert_cuda_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


⚠️ 조합 보정됨: 유사도 0.776
🔍 L5는 분류 대신 유사도 기반 추천으로 예측되었습니다 (유사도: 0.877)

🔎 유사한 품목 Top 5:

▶ 품명: STRAUB COUPLINGS PIPE COUPLING STRAUB GRIP TYPE   RUBBER EPDM SLEEVE 50A (유사도: 77.6%)
   🔹 P CODE: SCASS2P205

▶ 품명: STRAUB COUPLINGS PIPE COUPLING STRAUB GRIP TYPE   RUBBER EPDM SLEEVE 40A (유사도: 77.5%)
   🔹 P CODE: SCASS2P204

▶ 품명: STRAUB COUPLINGS PIPE COUPLING STRAUB GRIP TYPE   RUBBER EPDM SLEEVE 25A (유사도: 77.5%)
   🔹 P CODE: SCASS2P202

▶ 품명: STRAUB COUPLINGS PIPE COUPLING STRAUB GRIP TYPE   RUBBER EPDM SLEEVE 65A (유사도: 77.5%)
   🔹 P CODE: SCASS2P206

▶ 품명: STRAUB COUPLINGS PIPE COUPLING STRAUB GRIP TYPE   RUBBER EPDM SLEEVE 20A (유사도: 77.5%)
   🔹 P CODE: SCASS2P201

✅ 총 소요 시간: 5.2초


: 

: 

2nd 사용 방법 (정확도 떨어짐)

In [45]:
# step_1_loading.py
import os
import time
import joblib
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from ai_mode_module import set_globals

load_dir = "AI_Model_2"

print("📦 모델 및 데이터 로딩 중...")
start = time.time()
df_model = pd.read_pickle(f"{load_dir}/df_model.pkl")
vectorizer = joblib.load(f"{load_dir}/searchname_vectorizer.pkl")
valid_combinations = joblib.load(f"{load_dir}/valid_combinations.pkl")

set_globals(
    vec=vectorizer,
    df_model_param=df_model,
    prep=preprocessor,
    models=hierarchical_models,
    encoders=hierarchical_label_encoders,
    valid_combos=valid_combinations
)

# 계층 모델들
required_cols = ['L1 NAME', 'L2 NAME', 'L3 NAME', 'L4 NAME']
hierarchical_models = {}
hierarchical_label_encoders = {}

for col in required_cols:
    print(f"📂 {col} 로딩 중...")
    model = joblib.load(f"{load_dir}/{col}_model.pkl")
    vec = joblib.load(f"{load_dir}/{col}_vectorizer.pkl")
    enc = joblib.load(f"{load_dir}/{col}_label_encoder.pkl")
    hierarchical_models[col] = (model, vec)
    hierarchical_label_encoders[col] = enc

print(f"✅ 모델 로딩 완료! (소요 시간: {round(time.time() - start, 2)}초)\n")


📦 모델 및 데이터 로딩 중...
📂 L1 NAME 로딩 중...
📂 L2 NAME 로딩 중...
📂 L3 NAME 로딩 중...
📂 L4 NAME 로딩 중...
✅ 모델 로딩 완료! (소요 시간: 1.79초)



In [46]:
# step_2_preprocessor.py (SimplePreprocessor 클래스 정의 붙이기)
from collections import defaultdict
import re
from sklearn.feature_extraction.text import TfidfVectorizer

class SimplePreprocessor:
    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.stop_words = set(["OF", "FOR", "WITH", "AND", "THE", "A", "AN", "TO", "IN", "ON", "BY"])
        self.unit_dict = self._learn_units()
        self.synonyms = self._learn_synonyms()
        self.manual_synonyms = {
    "SET": ["SET", "SETS", "KIT", "KITS", "KITAGAWA", "KITGRACO", "LIGHTINGKIT"],
    "BOX": ["BOX", "BOXES", "BOXING", "CASE", "CASES", "CARTON"],
    "BOTTLE": ["BTL", "BTLS", "BOTTLE", "BOTTLES", "FLASK"],
    "SPOON": ["SPOON", "SPOONS", "TEASPOON", "TABLESPOON", "LADLE"],
    "GLASS": ["GLASS", "GLASSES", "GLASSWARE", "FIBERGLASS", "FIBREGLASS", "MARINEGLASS"],
    "JAR": ["JAR", "JARS", "JARLSBERG", "CANISTER", "CONTAINER"],
    "MUG": ["MUG", "MUGS", "CUP", "CUPS", "TUMBLER", "STEIN"],
    "TRAY": ["TRAY", "TRAYS", "PLATE", "PLATES", "DISH", "DISHES"],
    "TAPE": ["TAPE", "TAPES", "BAND", "STRAP", "STRAPS"],
    "HANDLE": ["HANDLE", "KNOB", "GRIP", "KNOBSET", "LEVER"],
    "VALVE": ["VALVE", "VALVES", "COCK", "STOPPER", "TAP"],
    "PIPE": ["PIPE", "TUBE", "TUBES", "HOSE", "DUCT", "CYLINDER"],
    "BLADE": ["BLADE", "BLADES", "KNIFE", "KNIVES", "CUTTER", "CUTTERS"],
    "COVER": ["COVER", "COVERS", "LID", "CAP", "SLEEVE", "SHIELD"],
    "CLOTH": ["CLOTH", "CLOTHES", "FABRIC", "TEXTILE", "RAG", "RAGS"],
    "GLOVE": ["GLOVE", "GLOVES", "MITT", "MITTS", "HANDGUARD"],
    "WIRE": ["WIRE", "CABLE", "CORD", "ROPE", "LINE"],
    "HELMET": ["HELMET", "CAP", "HEADGEAR", "HAT", "HARDHAT"],
    "GOGGLES": ["GOGGLES", "GLASSES", "SHADES", "VISOR", "PROTECTIVE EYEWEAR"],
    "BATTERY": ["BATTERY", "BATTERIES", "CELL", "ACCUMULATOR"],
    "PAINT": ["PAINT", "COATING", "SPRAY", "VARNISH", "ENAMEL"],
    "BAG": ["BAG", "BAGS", "SACK", "SACKS", "POUCH", "PACK"],
    "LABEL": ["LABEL", "TAG", "STICKER", "DECAL"],
    "CLEANER": ["CLEANER", "DETERGENT", "SOAP", "SANITIZER", "DISINFECTANT"],
    "FAN": ["FAN", "FANS", "VENTILATOR", "BLOWER"],
    "LIGHT": ["LIGHT", "LAMP", "LED", "BULB", "FIXTURE"],
    "PUMP": ["PUMP", "PUMPS", "DISPENSER", "INJECTOR"],
        }
        self.tfidf = self._learn_tfidf()

    def _learn_units(self):
        units = defaultdict(int)
        specs = self.df['L5 NAME (SPEC)'].dropna().astype(str)
        for spec in specs:
            for match in re.findall(r'(\d+)\s*([A-Z]{1,5})', spec.upper()):
                units[match[1]] += 1
        return dict(units)

    def _learn_synonyms(self):
        synonyms = {}
        for col in ['L1 NAME', 'L2 NAME', 'L3 NAME', 'L4 NAME']:
            if col in self.df.columns:
                for val in self.df[col].dropna().unique():
                    if '/' in val:
                        terms = [t.strip().upper() for t in val.split('/')]
                        for t in terms:
                            synonyms[t] = terms
        return synonyms

    def _learn_tfidf(self):
        texts = self.df[['L1 NAME', 'L2 NAME', 'L3 NAME', 'L4 NAME']].fillna('').agg(' '.join, axis=1)
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(texts)
        return dict(zip(vectorizer.get_feature_names_out(), tfidf_matrix.sum(axis=0).A1))

    def normalize_query(self, query: str):
        query = query.upper()
        query = re.sub(r"[^A-Z0-9\s/]", " ", query)  # 특수문자 제거
        for unit in self.unit_dict:
            query = re.sub(rf"(\d+)\s*{unit}", rf"\1{unit}", query)
        return query.strip()

    def expand_query(self, query: str):
        words = set()
        for token in query.split():
            token = token.strip()
            if len(token) < 2 or token in self.stop_words:
                continue
            words.add(token)
            if token in self.synonyms:
                words.update(self.synonyms[token])
            if token in self.manual_synonyms:
                words.update(self.manual_synonyms[token])
            if '/' in token:
                parts = token.split('/')
                words.update(parts)
                words.add(' '.join(parts))
        return ' '.join(sorted(words))

    def score_query(self, query: str):
        words = query.split()
        return sum(self.tfidf.get(w.lower(), 0) for w in words)

    def preprocess(self, query: str):
        norm = self.normalize_query(query)
        expanded = self.expand_query(norm)
        score = self.score_query(expanded)
        return {
            'original': query,
            'normalized': norm,
            'expanded': expanded,
            'tfidf_score': round(score, 4)
        }
    
    def preprocess_normalized(self, query: str):
        return self.normalize_query(query)

# ✅ 인스턴스 생성
preprocessor = SimplePreprocessor(df_model)


In [47]:
# step_3_functions.py
from difflib import get_close_matches
from sklearn.metrics.pairwise import cosine_similarity

"""
# 예측 함수
def predict_item_name(raw_name, verbose=True): ...

# 오타 보정 함수
def autocorrect_input(user_input, top_n=3, cutoff=0.6): ...

# 추천 함수들
def recommend_similar_items(input_name, top_n=5): ...
def recommend_global_similar_items(input_name, top_n=5): ...
def fallback_by_l1_l3(predicted, input_name, top_n=5): ...
def keyword_fallback_items(input_text, top_n=5): ...

# 조합 및 유효성 체크
def is_valid_combination(predicted, df_ref): ...
def suggest_similar_combination(predicted, df_ref, top_n=5): ...

# P CODE 생성
def generate_structured_pcode(predicted): ...
def generate_structured_pcode_based_on_similar(similar_items, fallback_predicted): ...
def recommend_similar_pcodes_detailed(new_pcode, top_n=5): ...
"""

# step_3_functions.py

from ai_mode_module import (
    predict_item_name,
    autocorrect_input,
    recommend_similar_items,
    recommend_global_similar_items,
    fallback_by_l1_l3,
    keyword_fallback_items,
    is_valid_combination,
    suggest_similar_combination,
    generate_structured_pcode,
    generate_structured_pcode_based_on_similar,
    recommend_similar_pcodes_detailed
)

# ✅ 여기서는 함수 '선언' 절대 금지
# 위에서 가져온 함수들을 그대로 사용하세요



In [48]:
def display_prediction(input_name):
    os.system('cls' if os.name == 'nt' else 'clear')
    print(f"\n🔍 입력 품명: {input_name}\n{'='*60}")

    # 1. 오타 보정
    corrected = autocorrect_input(input_name)
    if corrected != input_name.upper():
        print(f"🔧 오타 보정 적용됨: '{input_name}' → '{corrected}'")
    else:
        print(f"✅ 입력 그대로 사용됨: '{corrected}'")

    # 2. 예측
    predicted = predict_item_name(corrected)

    # 3. 예측 신뢰도 계산
    pred_string = f"{predicted['L1 NAME']} {predicted['L2 NAME']} {predicted['L3 NAME']} {predicted['L4 NAME']} {predicted['L5 NAME (SPEC)']}"
    input_vec = vectorizer.transform([corrected.upper()])
    pred_vec = vectorizer.transform([pred_string.upper()])
    sim_score = cosine_similarity(input_vec, pred_vec)[0][0]
    print(f"\n📊 예측 신뢰도: {round(sim_score * 100, 1)}%")

    # 4. 유효 조합 여부
    if not is_valid_combination(predicted, df_model):
        print("⚠️ 예측된 L1~L5 조합이 유효하지 않습니다. 유사 조합 추천:")
        print(suggest_similar_combination(predicted, df_model))
    else:
        print("✅ 유효한 L1~L5 조합")

    # 5. P CODE 확인/생성
    matched = df_model[
        (df_model["L1 NAME"] == predicted["L1 NAME"]) &
        (df_model["L2 NAME"] == predicted["L2 NAME"]) &
        (df_model["L3 NAME"] == predicted["L3 NAME"]) &
        (df_model["L4 NAME"] == predicted["L4 NAME"]) &
        (df_model["L5 NAME (SPEC)"] == predicted["L5 NAME (SPEC)"])
    ]

    if not matched.empty:
        print(f"🎯 추천 P CODE (기존): {matched.iloc[0]['P CODE']}")
    else:
        sim_items = recommend_similar_items(corrected)
        new_pcode = generate_structured_pcode_based_on_similar(sim_items, predicted)
        print(f"🆕 추천 P CODE (신규): {new_pcode}")
        print(recommend_similar_pcodes_detailed(new_pcode))

    # 6. 유사 품목 추천
    print("\n🌐 전역 유사 품목:")
    for rec in recommend_global_similar_items(corrected):
        print(f"▶ {rec['SIMILAR_ITEM_NAME']} ({rec['SIMILARITY_SCORE']}) → {rec['P CODE']}")

    print("\n📌 동일 구조 내 유사 품목:")
    for rec in recommend_similar_items(corrected):
        print(f"▶ {rec['SIMILAR_ITEM_NAME']} ({rec['SIMILARITY_SCORE']}) → {rec['P CODE']}")


In [49]:
def run_model():
    while True:
        user_input = input("\n🔍 품명을 입력하세요 ('exit' 종료 / 'clear' 초기화): ").strip()
        if user_input.lower() in ['exit', 'quit']:
            print("👋 종료합니다.")
            break
        elif user_input.lower() == "clear":
            os.system('cls' if os.name == 'nt' else 'clear')
            continue
        elif not user_input:
            print("❗ 입력이 비어 있습니다.")
            continue

        display_prediction(user_input)


In [50]:
import ai_mode_module as ai
print(ai.df_model is not None)  # ✅ True가 나와야 함


True


In [51]:
if __name__ == "__main__":
    run_model()



🔍 입력 품명: PIPE
✅ 입력 그대로 사용됨: 'PIPE'


c:\Users\정하민\Desktop\엠투코리아 모델\bert_cuda_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


🔍 L5는 유사도 기반 추천됨 (유사도: 0.812)

📊 예측 신뢰도: 0.0%
⚠️ 예측된 L1~L5 조합이 유효하지 않습니다. 유사 조합 추천:
       P CODE                                  SEARCH_NAME  SIMILARITY_SCORE
0  SGTBE1E007  SCREW / NUT NUT EYE NUTS EYE NUT STEEL  M30               1.0
1  SGTBE1E00A  SCREW / NUT NUT EYE NUTS EYE NUT STEEL  M48               1.0
2  SGTBE1E009  SCREW / NUT NUT EYE NUTS EYE NUT STEEL  M42               1.0
3  SGTBE1E008  SCREW / NUT NUT EYE NUTS EYE NUT STEEL  M36               1.0
4  SGTBE1E006  SCREW / NUT NUT EYE NUTS EYE NUT STEEL  M24               1.0


c:\Users\정하민\Desktop\엠투코리아 모델\bert_cuda_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


🔍 L5는 유사도 기반 추천됨 (유사도: 0.812)
🆕 추천 P CODE (신규): SCNUEYEY001
Empty DataFrame
Columns: []
Index: []

🌐 전역 유사 품목:
▶ HAND TOOL PIPE BENDER PIPE BENDERS BENDER PIPE FOR PIPE SIZE 30A HANDLE LENGTH 1800MM (0.497) → SCTMP1B401
▶ HAND TOOL PIPE BENDER PIPE BENDERS BENDER PIPE FOR PIPE SIZE 50A HANDLE LENGTH 2100MM (0.481) → SCTMP1B601
▶ HAND TOOL PIPE BENDER PIPE BENDERS BENDER PIPE FOR PIPE SIZE 40A HANDLE LENGTH 2000MM (0.48) → SCTMP1B501
▶ HAND TOOL PIPE BENDER PIPE BENDERS BENDER PIPE FOR PIPE SIZE 25A HANDLE LENGTH 1700MM (0.48) → SCTMP1B301
▶ HAND TOOL PIPE BENDER PIPE BENDERS BENDER PIPE FOR PIPE SIZE 20A HANDLE LENGTH 1500MM (0.48) → SCTMP1B201

📌 동일 구조 내 유사 품목:


...

c:\Users\정하민\Desktop\엠투코리아 모델\bert_cuda_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


🔍 L5는 유사도 기반 추천됨 (유사도: 1.0)
🆕 추천 P CODE (신규): RICABAGA001
Empty DataFrame
Columns: []
Index: []

🌐 전역 유사 품목:
▶ LIQUOR WHISKY BALLANTINES BALLANTINES FINEST 4Y GBX 1L (0.831) → PAWHB1BM01
▶ LIQUOR WHISKY BALLANTINES BALLANTINES 4Y 1L (0.831) → PAWHB1BF01
▶ LIQUOR WHISKY BALLANTINES BALLANTINES FINEST 1L (0.831) → PAWHB1BG01
▶ LIQUOR WHISKY BALLANTINES BALLANTINES 12Y GBX 1L (0.831) → PAWHB1B301
▶ LIQUOR WHISKY BALLANTINES BALLANTINES 30Y 1L (0.831) → PAWHB1BC01

📌 동일 구조 내 유사 품목:


c:\Users\정하민\Desktop\엠투코리아 모델\bert_cuda_env\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


🔍 L5는 유사도 기반 추천됨 (유사도: 1.0)
❗ 입력이 비어 있습니다.


KeyboardInterrupt: Interrupted by user